# Dolly Test


## Preamble


In [1]:
from pathlib import Path
import torch
from transformers import pipeline
from langchain.llms.base import LLM
from llama_index import SimpleDirectoryReader, LangchainEmbedding, GPTVectorStoreIndex, PromptHelper, LLMPredictor, ServiceContext
from llama_index.langchain_helpers.text_splitter import TokenTextSplitter
from llama_index.node_parser.simple import SimpleNodeParser
from langchain.embeddings.huggingface import HuggingFaceEmbeddings


/Users/cls/miniforge3/envs/trIAge/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Workflow


In [2]:
INPUT_FOLDER = "../data/"

index_files = list(Path(INPUT_FOLDER).glob("*"))

max_input_size = 2048
num_output = 256
max_chunk_overlap = 20
prompt_helper = PromptHelper(max_input_size, num_output, max_chunk_overlap)


pipe = pipeline("text-generation", model="databricks/dolly-v2-3b", trust_remote_code=True, torch_dtype=torch.bfloat16, device_map="auto")
embed_model = LangchainEmbedding(HuggingFaceEmbeddings())

class CustomLLM(LLM):
    model_name = "databricks/dolly-v2-3b"

    def _call(self, prompt, stop = None):
        response = pipe(prompt, max_new_tokens=num_output)[0]["generated_text"]
        return response

    @property
    def _identifying_params(self):
        return {"name_of_model": self.model_name}

    @property
    def _llm_type(self):
        return "custom"

# define our LLM
llm_predictor = LLMPredictor(llm=CustomLLM())

node_parser = SimpleNodeParser(text_splitter=TokenTextSplitter(chunk_size=512, chunk_overlap=max_chunk_overlap))
prompt_helper = PromptHelper(max_input_size, num_output, max_chunk_overlap)
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, embed_model=embed_model, prompt_helper=prompt_helper, node_parser=node_parser, chunk_size_limit=512)
# Load your data
documents = SimpleDirectoryReader(input_files=index_files).load_data()

index = GPTVectorStoreIndex.from_documents(documents, service_context=service_context)

query_engine = index.as_query_engine()

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cpu
Batches:   0%|          | 0/1 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Batches: 100%|██████████| 1/1 [00:01<00:00,  1.12s/it]
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 1346 tokens


In [5]:
query_engine.query("What is the title of the project?")


Batches: 100%|██████████| 1/1 [00:00<00:00,  4.71it/s]
INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 8 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 295 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens


Response(response='Documentation is a project to automatically generate documentation from the code of a project.', source_nodes=[NodeWithScore(node=Node(text='\n\nTarget Functionality\n\n- Support: answer questions based on documentation, generate explanatory code examples\n- Issue Quality Control: analyze issues for quality of description and reproducibility, provide feedback for improvement\n- Issue Triage: Automatically categorize issues by content (feature request, bug, support) and tag them, detect duplicates, prioritize issues (e.g. security issues), link related issues\n- Debugging: Analyze error causes, generate solution suggestions\n- Testing: automatic generation of test cases based on issues\n- Pull Request Review: Analysis of pull requests regarding code quality, code style, documentation, test coverage, etc.\n- Documentation: Generation of documentation proposals from the project context\n- Changelogs: Generation of changelogs from the project history\n\n\n', doc_id='849d

In [6]:
query_engine.query("What is the target functionality?")


Batches: 100%|██████████| 1/1 [00:00<00:00,  9.77it/s]
INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 6 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 553 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens


Response(response='The new version of the data generator would be compatible with the gpt-3.5-turbo model.', source_nodes=[NodeWithScore(node=Node(text='\n\nTarget Functionality\n\n- Support: answer questions based on documentation, generate explanatory code examples\n- Issue Quality Control: analyze issues for quality of description and reproducibility, provide feedback for improvement\n- Issue Triage: Automatically categorize issues by content (feature request, bug, support) and tag them, detect duplicates, prioritize issues (e.g. security issues), link related issues\n- Debugging: Analyze error causes, generate solution suggestions\n- Testing: automatic generation of test cases based on issues\n- Pull Request Review: Analysis of pull requests regarding code quality, code style, documentation, test coverage, etc.\n- Documentation: Generation of documentation proposals from the project context\n- Changelogs: Generation of changelogs from the project history\n\n\n', doc_id='849dab44-1d